_Neural Data Analysis_

Lecturer: Prof. Dr. Philipp Berens

Tutors: Sarah Strauss, Ziwei Huang

Summer term 2021

Name: FILL IN YOUR NAMES HERE

# Coding Lab 7



In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.optimize as opt
import scipy.io as io
import scipy as sp

mpl.rc("savefig", dpi=72)

import itertools

sns.set_style('whitegrid')
%matplotlib inline

## Task 1: Fit RF on simulated data

We will start  with toy data generated from an LNP model neuron to make sure everything works right. The model LNP neuron consists of one Gaussian linear filter, an exponential nonlinearity and a Poisson spike count generator. We look at it in discrete time with time bins of width $\delta t$. The model is:

$$
c_t \sim Poisson(r_t)\\
r_t = \exp(w^T s_t) \cdot \Delta t \cdot R
$$

Here, $c_t$ is the spike count in time window $t$ of length $\Delta t$, $s_t$ is the stimulus and $w$ is the receptive field of the neuron. The receptive field variable `w` is 15 × 15 pixels and normalized to $||w||=1$. A stimulus frame is a 15 × 15 pixel image, for which we use uncorrelated checkerboard noise. R can be used to bring the firing rate into the right regime (e.g. by setting $R=50$).   

For computational ease, we reformat the stimulus and the receptive field in a 225 by 1 array. The function ```sampleLNP``` can be used to generate data from this model. It returns a spike count vector `c` with samples from the model (dimensions: 1 by nT = $T/\Delta t$), a stimulus matrix `s` (dimensions: 225 × nT) and the mean firing rate `r` (dimensions: nT × 1). 

Here we assume that the receptive field influences the spike count instantaneously just as in the above equations. Implement a Maximum Likelihood approach to fit the receptive field. 

To this end simplify and implement the log-likelihood function $L(w)$ and its gradient $\frac{L(w)}{dw}$ with respect to $w$ (`logLikLnp`). The log-likelihood of the model is
$$L(w) = \log \prod_t \frac{r_t^{c_t}}{c_t!}\exp(-r_t).$$

Plot the true receptive field, a stimulus frame, the spike counts and the estimated receptive field. 



### Calculations
Simplify the log likelihood analytically and derive the analytical solution for the gradient.
*(2 pts)*



### Generate data

In [ ]:
def gen_gauss_rf(D,width, center=(0,0)):
    
    # generates a Gaussian blob as an image
    
    sz = (D-1)/2
    x, y = sp.mgrid[-sz: sz + 1, -sz: sz + 1]
    x = x + center[0]
    y = y + center[1]
    w = sp.exp(- (x ** 2/width + y ** 2 / width))
    w = w/np.sum(w.flatten())
    
    return w

Generate samples from an instantaneous LNP model neuron with receptive field kernel w.
*(1pt)*

In [ ]:
def sampleLnp(w, nT, dt, R, v):
    '''
    Generate samples from an instantaneous LNP model neuron with 
    receptive field kernel w.
    
    Arguments:
        w:  receptive field kernel
        nT: number of time steps
        dt: duration of a frame in s
        R:  rate parameter
        v:  variance of the stimulus ensemble
    
    Returns:
        c:  sampled spike counts in time bins
        r:  mean rate in time bins
        s:  stimulus frames used
        
    See equations in task description above for a precise definition
    of the individual parameters.
    '''
    
    np.random.seed(10)
    
    # ENTER YOUR CODE HERE
    
    return c, r, s

In [ ]:
D = 15     # number of pixels
nT = 1000  # number of time bins
dt = 0.1   # bins of 100 ms
R = 50     # firing rate in Hz 
v = 5      #stimulus variance

w = gen_gauss_rf(D,7,(1,1))
w = w.flatten()

c, r, s = sampleLnp(w, nT, dt, R, v)



Plot the responses of the cell.

In [ ]:
plt.figure(figsize=(10,4))

t = np.arange(0,T) * dt;



### Implementation 

Before you run your optimizer, make sure the gradient is correct. The helper function `check_grad` in `scipy.optimize` can help you do that. This package also has suitable functions for optimization. If you generate a large number of  samples, the fitted receptive field will look more similar to the true receptive field. With more samples, the optimization takes longer, however.

*(2 pts)*


In [ ]:
def logLikeLnp(x, c, s):
    '''
      Plot spike rasters for a single neuron sorted by condition

      f, df = logLikLnp(x,c,s)
       Implements the negative (!) log-likelihood of the LNP model and its
       gradient with respect to the receptive field w.

       x   current receptive field (225 x 1)
       c   spike counts (1 x T/dt)
       s   stimulus matrix (225 x T/dt)

       f   function value of the negative log likelihood at x (scalar)
       df  gradient of the negative log likelihood with respect to x (225 x 1)

    '''



    return f, df


Fit receptive field maximizing the log likelihood

In [ ]:
# enter optimization code here

In [ ]:
plt.figure(figsize=(10,5))

# enter comparison true/estimated here

## Task 2: Apply to real neuron

Download the dataset for this task from Ilias (`nda_ex_6_data.mat`). It contains a stimulus matrix (`s`) in the same format you used before and the spike times. In addition, there is an array called `trigger` which contains the times at which the stimulus frames were swapped.

* Generate an array of spike counts at the same temporal resolution as the stimulus frames
* Fit the receptive field with time lags of 0 to 4 frames. Fit them one lag at a time (the ML fit is very sensitive to the number of parameters estimated and will not produce good results if you fit the full space-time receptive field for more than two time lags at once). 
* Plot the resulting filters

*Grading: 2 pts*


In [3]:
var = io.loadmat('../data/nda_ex_6_data.mat')

# t contains the spike times of the neuron
t = var['DN_spiketimes'].flatten()    

# trigger contains the times at which the stimulus flipped
trigger = var['DN_triggertimes'].flatten()

# contains the stimulus movie with black and white pixels
s = var['DN_stim']
s = s.reshape((300,1500))
s = s[:,1:len(trigger)]


Create vector of spike counts

In [4]:
c, b = np.histogram(t,bins=trigger)

Fit receptive field for each frame separately

Plot the frames one by one

In [ ]:
plt.figure(figsize=(10,4))



## Task 3: Separate space/time components

The receptive field of the neuron can be decomposed into a spatial and a temporal component. Because of the way we computed them, both are independent and the resulting spatio-temporal component is thus called separable. As discussed in the lecture, you can use singular-value decomposition to separate these two: 

$$
W = u_1 s_1 v_1^T
$$

Here $u_1$ and $v_1$ are the singular vectors belonging to the 1st singular value $s_1$ and provide a long rank approximation of W, the array with all receptive fields. It is important that the mean is subtracted before computing the SVD.  

Plot the first temporal component and the first spatial component. You can use a Python implementation of SVD. The results can look a bit puzzling, because the sign of the components is arbitrary.

*Grading: 1 pts*

## Task 4: Regularized receptive field

As you can see, maximum likelihood estimation of linear receptive fields can be quite noisy, if little data is available. 

To improve on this, one can regularize the receptive field vector and a term to the cost function


$$
C(w) = L(w) + \alpha ||w||_p^2
$$

Here, the $p$ indicates which norm of $w$ is used: for $p=2$, this is shrinks all coefficient equally to zero; for $p=1$, it favors sparse solutions, a penality also known as lasso. Because the 1-norm is not smooth at zero, it is not as straightforward to implement "by hand". 

Use a toolbox with an implementation of the lasso-penalization and fit the receptive field. Possibly, you will have to try different values of the regularization parameter $\alpha$. Plot your estimates from above and the lasso-estimates. How do they differ? What happens when you increase or decrease $alpha$?

If you want to keep the Poisson noise model, you can use the implementation in [`pyglmnet`](https://pypi.python.org/pypi/pyglmnet). Otherwise, you can also resort to the linear model from `sklearn` which assumes Gaussian noise (which in my hands was much faster).

*Grading: 2 pts*


In [2]:
import sklearn

In [ ]:
sklearn.linear

In [ ]:
 # enter fitting code here
        
    
    

In [ ]:
plt.figure(figsize=(10,8))

